In [1]:
import pandas as pd
import xml.etree.ElementTree as et 
import xmltodict

input_path = './export.xml'

with open(input_path, 'r') as xml_file:
    input_data = xmltodict.parse(xml_file.read())

records_list = input_data['HealthData']['Record']

df = pd.DataFrame(records_list)

df['@type'].unique()

df

# convert to date fromat
format = '%Y-%m-%d %H:%M:%S %z'

df['@creationDate'] = pd.to_datetime(df['@creationDate'],
                                     format=format)
df['@startDate'] = pd.to_datetime(df['@startDate'],
                                  format=format)
df['@endDate'] = pd.to_datetime(df['@endDate'],
                                format=format)

# make dataframe with steps
step_counts = df[df['@type'] == 'HKQuantityTypeIdentifierStepCount']
 
# convert to numeric value
step_counts.loc[:, '@value'] = pd.to_numeric(
    step_counts.loc[:, '@value'])


# limit the devices in case of multiple devices
#df = df[df['@sourceName'] == 'iPhone XS']

step_counts_by_creation = step_counts.groupby('@creationDate').sum()

step_counts_by_creation





,@value
@creationDate,
2020-11-27 18:23:53+01:00,2
2020-11-27 18:50:15+01:00,22
2020-11-27 19:10:04+01:00,8
2020-11-27 19:20:21+01:00,20
2020-11-27 20:01:47+01:00,26
...,...
2021-02-04 13:44:54+01:00,122
2021-02-04 14:01:39+01:00,44
2021-02-04 14:11:44+01:00,119


# Create daily dataframe

In [3]:
by_day = step_counts_by_creation['@value'].resample('D').sum()

by_day



@creationDate
2020-11-27 00:00:00+01:00     890
2020-11-28 00:00:00+01:00    1874
2020-11-29 00:00:00+01:00    1259
2020-11-30 00:00:00+01:00    1503
2020-12-01 00:00:00+01:00    3869
                             ... 
2021-01-31 00:00:00+01:00    6121
2021-02-01 00:00:00+01:00    4703
2021-02-02 00:00:00+01:00    2179
2021-02-03 00:00:00+01:00    4059
2021-02-04 00:00:00+01:00     807
Freq: D, Name: @value, Length: 70, dtype: int64